In [1]:
import requests
import requests_cache
requests_cache.install_cache('cache')
import requests_html
import math
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
from pprint import pprint
import re
import traceback
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_colwidth", None)

In [2]:
showClosed = False
first = requests.get(f"https://givealittle.co.nz/search/api?category=health&showClosed={showClosed}").json()
total = first["total"]
results = first["results"]
last_page = math.ceil(total / len(results))
print(f"Last page: {last_page}")
for page in tqdm(range(2, last_page + 1)):
    page_results = requests.get(f"https://givealittle.co.nz/search/api?category=health&showClosed={showClosed}&p={page}").json()
    results.extend(page_results["results"])

Last page: 97


  0%|          | 0/96 [00:00<?, ?it/s]

In [3]:
df = pd.DataFrame(results)
df.uri = "https://givealittle.co.nz" + df.uri
df.actionUri = "https://givealittle.co.nz" + df.actionUri
df

,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri
0,Lachie Love needs your help!,Lachie has an extremely rare and aggressive br...,https://static.givealittle.co.nz/assets/hero/b...,https://givealittle.co.nz/cause/lachie-needs-y...,131736.99,0.0,160 days to go,Donate,https://givealittle.co.nz/donate/cause/lachie-...
1,Dean's story,Deans fight against the big C!!!!!,https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/dean-story,27618.00,0.0,2 days to go,Donate,https://givealittle.co.nz/donate/cause/dean-story
2,"LIFE SAVING SURGERY REQUIRED, ""URGENT""","""LIFE SAVING SURGERY"" only available in Germany.",https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/health-system-...,82645.50,180000.0,73 days to go,Donate,https://givealittle.co.nz/donate/cause/health-...
3,"Lisa, Dan & the Fong boys - treatment for Dan ...","Lisa, Dan & the Fong boys - treatment required...",https://static.givealittle.co.nz/assets/hero/f...,https://givealittle.co.nz/cause/lisa-dan-and-t...,56394.97,0.0,48 days to go,Donate,https://givealittle.co.nz/donate/cause/lisa-da...
4,Connor needs life saving treatment in Melbourne,Connor needs life saving treatment in Melbourne,https://static.givealittle.co.nz/assets/hero/3...,https://givealittle.co.nz/cause/connor-needs-l...,65580.21,0.0,317 days to go,Donate,https://givealittle.co.nz/donate/cause/connor-...
...,...,...,...,...,...,...,...,...,...
957,Move for the Mind,Mental Health Fundraiser,https://static.givealittle.co.nz/assets/hero/6...,https://givealittle.co.nz/fundraiser/move-for-...,0.00,0.0,174 days to go,Donate,https://givealittle.co.nz/donate/fundraiser/mo...
958,360 trail challenge for mental health,Join me on my 360 Trail Run for Mental Health ...,https://static.givealittle.co.nz/assets/hero/2...,https://givealittle.co.nz/fundraiser/360-trail...,0.00,0.0,98 days to go,Donate,https://givealittle.co.nz/donate/fundraiser/36...
959,Pounding the pavement for migraine,Participating in Auckland Marathon 2024 Migrai...,https://static.givealittle.co.nz/assets/hero/8...,https://givealittle.co.nz/fundraiser/pounding-...,0.00,3000.0,149 days to go,Donate,https://givealittle.co.nz/donate/fundraiser/po...
960,Promote Mental Health through Meditation,Promotion of Mental Wellbeing through training...,https://static.givealittle.co.nz/assets/hero/9...,https://givealittle.co.nz/event/meditation4wel...,145.00,500000.0,180 days to go,Check it out,https://givealittle.co.nz/event/meditation4wel...


In [4]:
s = requests_html.HTMLSession()

In [5]:
def extract(row):
    uri = row.uri
    try:
        row = row.to_dict()
        html = s.get(uri).html
        if "This campaign started on" not in html.text:
            start = ""
            end = ""
        else:
            start, end = re.search(r"This campaign started on (.*) and ends on (.*)\.", html.text).groups()
        question_link = html.find("a[href$=questions]", first=True)
        if question_link.text.strip() in ["Ask a question here", "Ask one here"]:
            n_questions = 0
        elif "One question has been asked already" in html.text:
            n_questions = 1
        else:
            n_questions = int(re.search("(\d+) questions have been asked already", html.text).group(1))
        use_of_funds = html.find("h4", containing="Use of funds", first=True)
        if use_of_funds:
            use_of_funds = use_of_funds.element.getnext().text_content()
        else:
            use_of_funds = ""
        whos_involved = html.find("h4", containing="Who's involved?", first=True)
        if whos_involved:
            whos_involved = whos_involved.element.getnext().text_content().strip()
        else:
            whos_involved = ""
        location = html.find("body > div > div > div.text-gray-600.pb-8.md\:pt-6 > div.w-full.max-w-6xl.mx-auto.px-6.space-y-2 > div.flex.pt-2.md\:pt-0 > div.w-full.md\:w-7\/12.lg\:w-2\/3.md\:shrink-0.space-y-6.divide-y.divide-gray-200 > div:nth-child(1) > p.mt-4", first=True)
        if location:
            location = location.text
        else:
            location = ""
        n_donors = html.find("body > div > div > div.text-gray-600.pb-8.md\:pt-6 > div.w-full.max-w-6xl.mx-auto.px-6.space-y-2 > div.flex.pt-2.md\:pt-0 > div.flex.flex-grow > div > div > div > div.font-serif > div.pt-1 > div > span", first=True)
        if n_donors:
            n_donors = int(n_donors.text)
        else:
            n_donors = ""
        row.update({
            "start": start,
            "end": end,
            "n_questions": n_questions,
            "location": location,
            "n_donors": n_donors,
            "description": html.find("div.break-words", first=True).text,
            "use_of_funds": use_of_funds,
            "whos_involved": whos_involved,
            "moderated": html.find("div.font-semibold")[-1].text
        })
        html = s.get(uri + "/updates").html
        updates = html.find("article")
        row.update({
            "n_updates": len(updates),
            "updates": "\n".join([update.text for update in updates])
        })
        if n_questions:
            html = s.get(uri + "/questions").html
            row["Q&A"] = html.find("body > div > div > div.w-full.max-w-6xl.mx-auto.px-6.py-6.text-gray-600.space-y-4 > div > div > div", first=True).text
        return row
    except Exception as e:
        print(f"{e} when trying to extract {uri}")
        print(traceback.format_exc())

df_extracted = pd.json_normalize(df.progress_apply(extract, axis=1))
df_extracted

  0%|          | 0/962 [00:00<?, ?it/s]

,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri,start,end,n_questions,location,n_donors,description,use_of_funds,whos_involved,moderated,n_updates,updates,Q&A
0,Lachie Love needs your help!,Lachie has an extremely rare and aggressive br...,https://static.givealittle.co.nz/assets/hero/b...,https://givealittle.co.nz/cause/lachie-needs-y...,131736.99,0.0,160 days to go,Donate,https://givealittle.co.nz/donate/cause/lachie-...,6 Jan 2023,11 Dec 2024,5,Auckland,1554,Lachie is 8. He has a sparkle in his eyes & an...,Any donations will be used to alleviate the fi...,Created by\r\nNikki Donkin \n\n\n\n\r\n ...,Page Moderated,13,"Update!\n31 May 2024\nHey everyone, long time ...","on 16 Jan 2023\nKylie asks\nHi, my name is Kyl..."
1,Dean's story,Deans fight against the big C!!!!!,https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/dean-story,27618.00,0.0,2 days to go,Donate,https://givealittle.co.nz/donate/cause/dean-story,6 Jul 2023,6 Jul 2024,1,Nationwide,1478,On January 10th 2020 Dean and his family's liv...,Update 27/9: Funds raised will go towards supp...,Created by\r\nFiona Boniface \n\n\n\n\r\n ...,Page Moderated,2,RIP Dean\n11 October 2023\nThank you so much t...,"on 25 Sep 2023\nKylie asks\nHi, I work for a c..."
2,"LIFE SAVING SURGERY REQUIRED, ""URGENT""","""LIFE SAVING SURGERY"" only available in Germany.",https://static.givealittle.co.nz/assets/hero/a...,https://givealittle.co.nz/cause/health-system-...,82645.50,180000.0,73 days to go,Donate,https://givealittle.co.nz/donate/cause/health-...,29 Jul 2023,15 Sep 2024,3,"Napier, Hawke's Bay",1023,"Trinity is just 18yrs old, & needs URGENT life...","Initially, Costs to have diagnostic tests and ...","Created by, and paying to a verified bank acco...",Page Moderated,23,EXTRA TIME\n15 June 2024\nWhilst we are gettin...,"on 8 Apr 2024\nJune asks\nHi Lynn,\nI note abo..."
3,"Lisa, Dan & the Fong boys - treatment for Dan ...","Lisa, Dan & the Fong boys - treatment required...",https://static.givealittle.co.nz/assets/hero/f...,https://givealittle.co.nz/cause/lisa-dan-and-t...,56394.97,0.0,48 days to go,Donate,https://givealittle.co.nz/donate/cause/lisa-da...,21 Jun 2024,21 Aug 2024,0,Canterbury,1029,"Hi all,\nAs you may or may not know, Dan Fong,...",Various cancer treatment including chemotherap...,Created by\r\nJessica Mizzi \n\n\n\n\r\n ...,Page Moderated,1,No updates have been posted,NaN
4,Connor needs life saving treatment in Melbourne,Connor needs life saving treatment in Melbourne,https://static.givealittle.co.nz/assets/hero/3...,https://givealittle.co.nz/cause/connor-needs-l...,65580.21,0.0,317 days to go,Donate,https://givealittle.co.nz/donate/cause/connor-...,17 May 2024,17 May 2025,1,"Christchurch, Canterbury",972,Connor is 7 years old and was diagnosed with A...,Apply for high health funding for treatment fo...,Created by\r\nLisa Chalmers \n\n\n\n\r\n ...,Page Moderated,5,Connor has received his 2000th bead\n29 June 2...,"on 28 May 2024\nKylie asks\nHi, I work for a c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,Move for the Mind,Mental Health Fundraiser,https://static.givealittle.co.nz/assets/hero/6...,https://givealittle.co.nz/fundraiser/move-for-...,0.00,0.0,174 days to go,Donate,https://givealittle.co.nz/donate/fundraiser/mo...,25 Jun 2024,25 Dec 2024,0,"Waikato, Waikato District",,"The annual charity event, taking place on the ...",,Created by\r\nAllaynah Hill \n\n\n\n\n\n\r\n ...,Page Moderated,1,No updates have been posted,NaN
958,360 trail challenge for mental health,Join me on my 360 Trail Run for Mental Health ...,https://static.givealittle.co.nz/assets/hero/2...,https://givealittle.co.nz/fundraiser/360-trail...,0.00,0.0,98 days to go,Donate,https://givealittle.co.nz/donate/fundraiser/36...,1 Jul 2024,10 Oct 2024,0,"Christchurch, Canterbury",,"Hi, I’m Jason, and I’m committed to raising aw...",,Created by\r\nJason Shaskey \n\n\n\n\n\n\r\n ...,Page Moderated,1,No upd

In [6]:
df_extracted.to_csv("givealittle_health.csv", index=False)